<a href="https://colab.research.google.com/github/dilanHewawitharana/Intro-to-Deep-Learning-with-PyTorch/blob/master/02_insurance_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.utils.colab.set_colab_file_id('11dezh1XoSevjh7ED7q53tUvN5h5SlL87')

# Insurance cost prediction using linear regression

Make a submisson here: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-2-train-your-first-model

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from [Kaggle](https://www.kaggle.com/mirichoi0218/insurance).


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lessons. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ai/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ai/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ai/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [90]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
!pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy matplotlib pandas torch torchvision torchaudio

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [91]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [92]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [93]:
DATASET_URL = "https://hub.jovian.ml/wp-content/uploads/2020/05/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

Using downloaded and verified file: ./insurance.csv


To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [94]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [95]:
your_name = "dilan"

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [96]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [97]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,charges
12,23,male,36.12000,0,no,1972.990440
306,28,female,28.87500,2,no,21791.884820
318,44,female,29.02725,0,no,8014.890114
815,20,female,33.03300,0,no,2028.163752
157,18,male,26.43375,0,yes,16759.634670


Let us answer some basic questions about the dataset. 


**Q: How many rows does the dataset have?**

In [98]:
num_rows = len(dataframe)
print(num_rows)

1271


**Q: How many columns doe the dataset have**

In [99]:
num_cols = dataframe.shape[1]
print(num_cols)

6


**Q: What are the column titles of the input variables?**

In [100]:
input_cols = ["age", "sex", "bmi", "children", "smoker", "charges"]

**Q: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [101]:
categorical_cols = ["sex", "smoker"]

**Q: What are the column titles of output/target variable(s)?**

In [102]:
output_cols = ["charges"]

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ml/aakashns/dataviz-cheatsheet

In [103]:
# Write your answer here
charge_col = dataframe["charges"]
print("min_value : {}".format(min(charge_col)))
print("max_value : {}".format(max(charge_col)))
print("avg_value : {}".format(sum(charge_col)/len(charge_col)))

min_value : 1211.623812
max_value : 68872.06225080001
avg_value : 14422.344658803895


Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [104]:
!pip install jovian --upgrade -q

In [105]:
import jovian

In [106]:
jovian.commit(project=project_name)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/dilanhewawitharana/02-insurance-linear-regression


'https://jovian.ai/dilanhewawitharana/02-insurance-linear-regression'

## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [107]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy(dtype='float32')
    targets_array = dataframe1[output_cols].to_numpy(dtype='float32')
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [108]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[2.3000000e+01, 1.0000000e+00, 3.6119999e+01, 0.0000000e+00,
         0.0000000e+00, 1.9729905e+03],
        [2.8000000e+01, 0.0000000e+00, 2.8875000e+01, 2.0000000e+00,
         0.0000000e+00, 2.1791885e+04],
        [4.4000000e+01, 0.0000000e+00, 2.9027250e+01, 0.0000000e+00,
         0.0000000e+00, 8.0148901e+03],
        ...,
        [3.5000000e+01, 1.0000000e+00, 3.8503502e+01, 1.0000000e+00,
         1.0000000e+00, 4.2956219e+04],
        [5.9000000e+01, 0.0000000e+00, 3.2917500e+01, 0.0000000e+00,
         0.0000000e+00, 1.3631954e+04],
        [6.2000000e+01, 0.0000000e+00, 3.4613251e+01, 3.0000000e+00,
         0.0000000e+00, 1.6861168e+04]], dtype=float32), array([[ 1972.9905],
        [21791.885 ],
        [ 8014.89  ],
        ...,
        [42956.22  ],
        [13631.954 ],
        [16861.168 ]], dtype=float32))

**Q: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [109]:
inputs = torch.from_numpy(inputs_array)
targets = torch.from_numpy(targets_array)

In [110]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [111]:
dataset = TensorDataset(inputs, targets)

**Q: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets.**

In [112]:
val_percent = 0.15 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = random_split(dataset, [train_size, val_size]) # Use the random_split function to split dataset into 2 parts of the desired length

Finally, we can create data loaders for training & validation.

**Q: Pick a batch size for the data loader.**

In [113]:
batch_size = 128

In [114]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [115]:
for xb, yb in train_loader:
    print("inputs:", len(xb))
    print("targets:", len(yb))
    break

inputs: 128
targets: 128


Let's save our work by committing to Jovian.

In [116]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Committed successfully! https://jovian.ai/dilanhewawitharana/02-insurance-linear-regression


'https://jovian.ai/dilanhewawitharana/02-insurance-linear-regression'

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [117]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [275]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size) # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb) # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss_fn = F.l1_loss
        loss = loss_fn(out, targets)                       # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss_fn = F.l1_loss
        loss = loss_fn(out, targets)                          # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [276]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [277]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.3606, -0.2854, -0.0319,  0.2017,  0.2416,  0.2727]],
        requires_grad=True), Parameter containing:
 tensor([0.0862], requires_grad=True)]

One final commit before we train the model.

In [278]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Committed successfully! https://jovian.ai/dilanhewawitharana/02-insurance-linear-regression


'https://jovian.ai/dilanhewawitharana/02-insurance-linear-regression'

## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [279]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [280]:
result = evaluate(model, val_loader) # Use the the evaluate function
print(result)

{'val_loss': 11320.24609375}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [281]:
epochs = 500
lr = 1e-8
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 10922.4824
Epoch [40], val_loss: 10524.7246
Epoch [60], val_loss: 10126.4053
Epoch [80], val_loss: 9727.7598
Epoch [100], val_loss: 9330.0273
Epoch [120], val_loss: 8930.7852
Epoch [140], val_loss: 8533.4707
Epoch [160], val_loss: 8136.3535
Epoch [180], val_loss: 7738.4707
Epoch [200], val_loss: 7339.8906
Epoch [220], val_loss: 6942.0889
Epoch [240], val_loss: 6544.6973
Epoch [260], val_loss: 6146.2197
Epoch [280], val_loss: 5747.3809
Epoch [300], val_loss: 5349.3896
Epoch [320], val_loss: 4951.5654
Epoch [340], val_loss: 4553.3604
Epoch [360], val_loss: 4156.0879
Epoch [380], val_loss: 3759.0256
Epoch [400], val_loss: 3359.7529
Epoch [420], val_loss: 2961.2573
Epoch [440], val_loss: 2564.0945
Epoch [460], val_loss: 2166.0801
Epoch [480], val_loss: 1767.9097
Epoch [500], val_loss: 1370.4001


In [282]:
epochs = 500
lr = 1e-8
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 972.2659
Epoch [40], val_loss: 573.9235
Epoch [60], val_loss: 176.0812
Epoch [80], val_loss: 9.6654
Epoch [100], val_loss: 9.5484
Epoch [120], val_loss: 9.5531
Epoch [140], val_loss: 9.5838
Epoch [160], val_loss: 9.5511
Epoch [180], val_loss: 9.6189
Epoch [200], val_loss: 9.5550
Epoch [220], val_loss: 9.6323
Epoch [240], val_loss: 9.6160
Epoch [260], val_loss: 9.5609
Epoch [280], val_loss: 9.5828
Epoch [300], val_loss: 9.6339
Epoch [320], val_loss: 9.5630
Epoch [340], val_loss: 9.6702
Epoch [360], val_loss: 9.6529
Epoch [380], val_loss: 9.5387
Epoch [400], val_loss: 9.6036
Epoch [420], val_loss: 9.5276
Epoch [440], val_loss: 9.5594
Epoch [460], val_loss: 9.5274
Epoch [480], val_loss: 9.5182
Epoch [500], val_loss: 9.5993


In [283]:
epochs = 500
lr = 1e-8
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 9.5500
Epoch [40], val_loss: 9.5551
Epoch [60], val_loss: 9.6056
Epoch [80], val_loss: 9.5222
Epoch [100], val_loss: 9.5198
Epoch [120], val_loss: 9.5030
Epoch [140], val_loss: 9.5086
Epoch [160], val_loss: 9.5578
Epoch [180], val_loss: 9.5029
Epoch [200], val_loss: 9.5196
Epoch [220], val_loss: 9.5045
Epoch [240], val_loss: 9.5461
Epoch [260], val_loss: 9.4956
Epoch [280], val_loss: 9.5756
Epoch [300], val_loss: 9.5460
Epoch [320], val_loss: 9.5661
Epoch [340], val_loss: 9.5202
Epoch [360], val_loss: 9.5242
Epoch [380], val_loss: 9.5287
Epoch [400], val_loss: 9.5213
Epoch [420], val_loss: 9.5268
Epoch [440], val_loss: 9.5215
Epoch [460], val_loss: 9.5634
Epoch [480], val_loss: 9.5265
Epoch [500], val_loss: 9.5067


In [284]:
epochs = 500
lr = 1e-8
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 9.4752
Epoch [40], val_loss: 9.4744
Epoch [60], val_loss: 9.5665
Epoch [80], val_loss: 9.5326
Epoch [100], val_loss: 9.4749
Epoch [120], val_loss: 9.5731
Epoch [140], val_loss: 9.6171
Epoch [160], val_loss: 9.4787
Epoch [180], val_loss: 9.5000
Epoch [200], val_loss: 9.4587
Epoch [220], val_loss: 9.5047
Epoch [240], val_loss: 9.4475
Epoch [260], val_loss: 9.4751
Epoch [280], val_loss: 9.4722
Epoch [300], val_loss: 9.4508
Epoch [320], val_loss: 9.5059
Epoch [340], val_loss: 9.5365
Epoch [360], val_loss: 9.4566
Epoch [380], val_loss: 9.4411
Epoch [400], val_loss: 9.5250
Epoch [420], val_loss: 9.5220
Epoch [440], val_loss: 9.5253
Epoch [460], val_loss: 9.4453
Epoch [480], val_loss: 9.4532
Epoch [500], val_loss: 9.5696


In [285]:
epochs = 500
lr = 1e-8
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 9.4927
Epoch [40], val_loss: 9.4311
Epoch [60], val_loss: 9.5904
Epoch [80], val_loss: 9.4254
Epoch [100], val_loss: 9.4329
Epoch [120], val_loss: 9.4610
Epoch [140], val_loss: 9.5488
Epoch [160], val_loss: 9.4666
Epoch [180], val_loss: 9.4638
Epoch [200], val_loss: 9.4241
Epoch [220], val_loss: 9.4827
Epoch [240], val_loss: 9.4561
Epoch [260], val_loss: 9.4327
Epoch [280], val_loss: 9.4797
Epoch [300], val_loss: 9.4270
Epoch [320], val_loss: 9.4327
Epoch [340], val_loss: 9.4156
Epoch [360], val_loss: 9.4218
Epoch [380], val_loss: 9.4243
Epoch [400], val_loss: 9.5239
Epoch [420], val_loss: 9.5169
Epoch [440], val_loss: 9.4912
Epoch [460], val_loss: 9.5016
Epoch [480], val_loss: 9.5010
Epoch [500], val_loss: 9.4488


**Q: What is the final validation loss of your model?**

In [ ]:
val_loss = ???

Let's log the final validation loss to Jovian and commit the notebook

In [ ]:
jovian.log_metrics(val_loss=val_loss)

In [ ]:
jovian.commit(project=project_name, environment=None)

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q: Complete the following function definition to make predictions on a single input**

In [ ]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = ???                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [ ]:
input, target = val_ds[0]
predict_single(input, target, model)

In [ ]:
input, target = val_ds[10]
predict_single(input, target, model)

In [ ]:
input, target = val_ds[23]
predict_single(input, target, model)

Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to replicate this notebook for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patterns in machine learning from problem-specific details.You can use one of these starer notebooks (just change the dataset):

- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog. 

Don't forget to share your work on the forum: https://jovian.ai/forum/t/linear-regression-and-logistic-regression-notebooks-and-blog-posts/14039

In [ ]:
jovian.commit(project=project_name, environment=None)
jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "aakashns/02-insurance-linear-regression" on https://jovian.ai/
[jovian] Uploading notebook..
[jovian] Committed successfully! https://jovian.ai/aakashns/02-insurance-linear-regression


<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
